In [1]:
using Distributed 
addprocs(19)
@everywhere using DelimitedFiles
@everywhere using Random, Distributions;  Random.seed!();
@everywhere using Distributed

In [2]:
@everywhere using Statistics
@everywhere using SpecialFunctions
@everywhere using QuadGK
@everywhere using DataStructures
@everywhere using DelimitedFiles
@everywhere using Roots

In [3]:
#SISk
@everywhere function SISk(x,parms)
    (S,I) = x
    (μ,β,κ,γ,N) = parms
    birth = μ*N
    death = μ*S
    infection = β*S*I/N + κ*S
    recovery = γ*I
    death_INF = μ*I
    event_weight = [birth death infection recovery death_INF]
    event_shift = [[1 0]; [-1 0]; [-1 1]; [1 -1]; [0 -1]]
    return event_weight, event_shift
end

In [4]:
@everywhere function GSP(F::Function,state,parms,time_end)
    time_points = collect(0:2:floor(time_end))
    I_vec_out = zeros(length(time_points))
    I_vec_out[1] = state[2]
    t = 0 #initial time is zero
    a = []    
    enact_event = zeros(0)

    while t<(time_points[end] +2)
        
        # to build time interval
        t1 = t
        
        ###############
        ## time step ##
        ###############
        a = F(state,parms)

        cumul_a = cumsum(a[1][1,:])
        num_o_events = length(a[1][1,:])
        a0 = cumul_a[num_o_events]
        #random numbers
        r1 = rand()
        r2 = rand()
        dtau = (1/a0)*log(1/r1)
        t += dtau
        
        ################################
        ### Append the output vector ###
        ################################

        timeloc =  findall(x->(x>t1 && x<=t), time_points)
        I_vec_out[timeloc] = repeat([state[2]],inner=1,outer=length(timeloc))
        

        ##############################
        ### determine event ##########
        ##############################
        marker = r2*a0
        i=1
        while i<=num_o_events
            if marker<=cumul_a[i]
                enact_event = i
                i = num_o_events+1
            end
            i+=1
        end


        #######################
        ### change the state ##
        #######################
        state = state + (a[2][enact_event,:])' #these are the state alterations defined by the function


    end #end while

    return time_points, I_vec_out
end 

In [7]:
###################
### run GSP sim ###
###################
@everywhere using Printf
@everywhere function run_GSP(kval_u,Nval_u)
    
    
#     kmat = readdlm("/C:/Users/WeNieds/Repos/SISkappa/data/in/SIS_IC_k.csv", ',', Float64)
    kmat = readdlm("C:/Users/WeNieds/Repos/SIS_externalRes/makeSimData/in/SIS_IC_k_test.csv", ',', Float64)
    bleck = 18
    krow_util = findall(x->x==round(kval_u,digits=bleck), round.(kmat,digits=bleck))
    krow = krow_util[1][1]

    Nmat = readdlm("C:/Users/WeNieds/Repos/SIS_externalRes/makeSimData/in/SIS_IC_N_test.csv", ',', Float64)
    blah = 9
    Ncol_util = findall(x->x==round(Nval_u,digits=blah), round.(Nmat,digits=blah))
    Ncol = Ncol_util[1][2] # returns the correct column

    ICmat = readdlm("C:/Users/WeNieds/Repos/SIS_externalRes/makeSimData/in/SIS_IC_test.csv", ',', Float64)
    
    I_IC = ICmat[krow,Ncol]

    S_IC = floor(Nval_u) - I_IC
    u0 = [S_IC I_IC]
    
    # set parameters find equilibrium
    param=[5e-5 0.5 kval_u 1 Nval_u]
    

    # Make initial conditions
     (μ,β,κ,γ,N) = param

    # run the Gillespie simulation
#     simdat = GSP(SISk,u0,param,2000);
    simdat = GSP(SISk,u0,param,10);
    n_len = length(simdat[1])
    kap_vec = zeros(n_len)
    kap_vec[1] = κ
    N_vec = zeros(n_len)
    N_vec[1] = N
    
    # Make strings for file names
    ku = @sprintf("%.3E", κ)
    ku = replace(ku, "." => "p")
    ku = replace(ku, "-" => "m")
    ku = replace(ku, "+" => "")
    
    Nu = @sprintf("%.3E", N)
    Nu = replace(Nu, "." => "p")
    Nu = replace(Nu, "-" => "m")
    Nu = replace(Nu, "+" => "")
    
    simdat_util = [simdat[1] simdat[2] kap_vec N_vec] 
    flname = "SISk_keq_"*ku*"_Neq_"*Nu
    writedlm(flname,  simdat_util, ',') 

    return nothing
end

In [8]:
# make sample list of kappa values
# kappa_list_util = 10.0 .^(range(-7,stop=-4,length=30))
kappa_list_util = 10.0 .^(range(-7,stop=-4,length=2))
kappa_list = zeros(0)

# make sample list of N values
# N_list_util = 10.0 .^(range(2,stop=5,length=30))
N_list_util = 10.0 .^(range(2,stop=5,length=2))
N_list = zeros(0)

for i in kappa_list_util
    for j in N_list_util
        # make lists for pmap
        append!(kappa_list, i)
        append!(N_list, j)
    end
end
pmap(run_GSP,kappa_list,N_list)

4-element Array{Nothing,1}:
 nothing
 nothing
 nothing
 nothing

      From worker 13:	ErrorException("Process(13) - Invalid connection credentials sent by remote.")CapturedException(ErrorException("Process(13) - Invalid connection credentials sent by remote."), Any[(error(::String) at error.jl:33, 1), (process_hdr(::Sockets.TCPSocket, ::Bool) at process_messages.jl:273, 1), (message_handler_loop(::Sockets.TCPSocket, ::Sockets.TCPSocket, ::Bool) at process_messages.jl:167, 1), (process_tcp_streams(::Sockets.TCPSocket, ::Sockets.TCPSocket, ::Bool) at process_messages.jl:142, 1), ((::Distributed.var"#99#100"{Sockets.TCPSocket,Sockets.TCPSocket,Bool})() at task.jl:356, 1)])
      From worker 13:	Process(13) - Unknown remote, closing connection.
      From worker 13:	ErrorException("Process(13) - Invalid connection credentials sent by remote.")CapturedException(ErrorException("Process(13) - Invalid connection credentials sent by remote."), Any[(error(::String) at error.jl:33, 1), (process_hdr(::Sockets.TCPSocket, ::Bool) at process_messages.jl:273, 1), 